In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00


In [ ]:
# Uninstall the current version of transformers
!pip uninstall transformers -y

# Install a compatible version (e.g., 4.26.0, you might need to experiment with versions)
!pip install transformers==4.26.0

Found existing installation: transformers 4.54.0
Uninstalling transformers-4.54.0:
  Successfully uninstalled transformers-4.54.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.26.0 which is incompatible.


#Checker

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import numpy as np
from sklearn.metrics import classification_report
import os
from underthesea import text_normalize, word_tokenize


In [ ]:

# 1. Tắt WandB
os.environ["WANDB_DISABLED"] = "true"

# 2. Load dữ liệu CSV
df = pd.read_csv('/content/drive/MyDrive/NLP/Phong/data_11.csv', encoding='utf-8-sig', on_bad_lines='skip')

# 3. Làm sạch và chuẩn hóa dữ liệu
df = df.dropna(subset=["prompt", "response", "response_label"])
df = df[df["response_label"].isin(["safe", "unsafe"])]  # Chỉ lấy safe/unsafe

# Danh sách từ dừng tiếng Việt
stop_words = {
    'là', 'rằng', 'với', 'về', 'cho', 'trong', 'trên', 'dưới', 'và', 'thì', 'nhưng', 'mà',
    'cũng', 'nếu', 'khi', 'tại', 'được', 'bị', 'có', 'không', 'đã', 'sẽ', 'này', 'kia'
}

# Hàm chuẩn hóa và loại bỏ ký tự đặc biệt
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    # Chuyển về chữ thường và chuẩn hóa tiếng Việt
    text = text_normalize(text.lower())
    # Loại bỏ ký tự đặc biệt "[]"
    text = text.replace("[", "").replace("]", "")
    # Loại bỏ các ký tự không phải chữ cái (giữ khoảng trắng và chữ)
    text = ''.join(c for c in text if c.isalpha() or c.isspace())
    # Tokenize
    tokens = word_tokenize(text, format="text")
    # Loại bỏ từ dừng
    tokens = [word for word in tokens.split() if word not in stop_words]
    return ' '.join(tokens)

# Áp dụng chuẩn hóa
df["prompt"] = df["prompt"].apply(preprocess_text)
df["response"] = df["response"].apply(preprocess_text)

# Chuẩn bị đầu vào
df["text"] = "[CLS] " + df["prompt"] + " [SEP] " + df["response"] + " [SEP]"
df["label"] = df["response_label"].map({"unsafe": 0, "safe": 1})

# Kiểm tra cân bằng
print("Số lượng mẫu - Unsafe:", len(df[df["response_label"] == "unsafe"]))
print("Số lượng mẫu - Safe:", len(df[df["response_label"] == "safe"]))

# 4. Tách train/val
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text"].tolist(), df["label"].tolist(), test_size=0.1, random_state=42, stratify=df["label"]
)

# 5. Load tokenizer và mô hình PhoBERT
model_checkpoint = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

# 6. Dataset Class
class SafeConvDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoded = self.tokenizer.encode_plus(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in encoded.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = SafeConvDataset(train_texts, train_labels, tokenizer)
val_dataset = SafeConvDataset(val_texts, val_labels, tokenizer)

# 7. Load mô hình PhoBERT với lớp phân loại nhị phân
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
model.to(device)

for param in model.roberta.encoder.layer[:-4].parameters():
    param.requires_grad = False

# 8. Thiết lập huấn luyện
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/NLP/Tieu_Luan/model_results",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.2,
    evaluation_strategy ="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_total_limit=2,
    report_to="none"
)

# 9. Metric đánh giá
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = (preds == labels).mean()
    return {"accuracy": acc}

# 10. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# 11. Huấn luyện trên GPU
trainer.train()

# 12. Lưu mô hình
model.save_pretrained("/content/drive/MyDrive/NLP/Tieu_Luan/model_results")
tokenizer.save_pretrained("/content/drive/MyDrive/NLP/Tieu_Luan/model_results")

# 13. Đánh giá và báo cáo
eval_result = trainer.evaluate()
print("🔍 Evaluation Results:", eval_result)

preds_output = trainer.predict(val_dataset)
y_true = preds_output.label_ids
y_pred = np.argmax(preds_output.predictions, axis=1)
print("\n📊 Classification Report:\n", classification_report(y_true, y_pred, target_names=["unsafe", "safe"]))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--vinai--phobert-base/snapshots/c1e37c5c86f918761049cef6fa216b4779d0d01d/config.json
Model config RobertaConfig {
  "_name_or_path": "vinai/phobert-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddi

Số lượng mẫu - Unsafe: 274
Số lượng mẫu - Safe: 228


loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--vinai--phobert-base/snapshots/c1e37c5c86f918761049cef6fa216b4779d0d01d/vocab.txt
loading file bpe.codes from cache at /root/.cache/huggingface/hub/models--vinai--phobert-base/snapshots/c1e37c5c86f918761049cef6fa216b4779d0d01d/bpe.codes
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--vinai--phobert-base/snapshots/c1e37c5c86f918761049cef6fa216b4779d0d01d/config.json
Model config RobertaConfig {
  "_name_or_path": "vinai/phobert-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer

Epoch,Training Loss,Validation Loss,Accuracy
1,0.707500,0.670213,0.549020
2,0.641900,0.579419,0.705882
3,0.488400,0.510389,0.784314
4,0.366800,0.505867,0.764706
5,0.201000,0.518177,0.764706
6,0.125800,0.625825,0.784314
7,0.058500,0.715808,0.803922
8,0.072000,0.788933,0.803922
9,0.042700,0.806149,0.823529
10,0.007200,0.827359,0.823529


***** Running Evaluation *****
  Num examples = 51
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/NLP/Tieu_Luan/model_results/checkpoint-29
Configuration saved in /content/drive/MyDrive/NLP/Tieu_Luan/model_results/checkpoint-29/config.json
Model weights saved in /content/drive/MyDrive/NLP/Tieu_Luan/model_results/checkpoint-29/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/NLP/Tieu_Luan/model_results/checkpoint-29/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/NLP/Tieu_Luan/model_results/checkpoint-29/special_tokens_map.json
added tokens file saved in /content/drive/MyDrive/NLP/Tieu_Luan/model_results/checkpoint-29/added_tokens.json
Deleting older checkpoint [/content/drive/MyDrive/NLP/Tieu_Luan/model_results/checkpoint-116] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 51
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/NLP/Tieu_Luan/model_results/checkpoint-58
Con

***** Running Prediction *****
  Num examples = 51
  Batch size = 16


🔍 Evaluation Results: {'eval_loss': 0.5058667659759521, 'eval_accuracy': 0.7647058823529411, 'eval_runtime': 1.6303, 'eval_samples_per_second': 31.283, 'eval_steps_per_second': 2.454, 'epoch': 10.0}

📊 Classification Report:
               precision    recall  f1-score   support

      unsafe       0.81      0.75      0.78        28
        safe       0.72      0.78      0.75        23

    accuracy                           0.76        51
   macro avg       0.76      0.77      0.76        51
weighted avg       0.77      0.76      0.77        51



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from underthesea import text_normalize, word_tokenize

# Load mô hình đã huấn luyện (chuyển sang CPU)
model_path = "/content/drive/MyDrive/NLP/Tieu_Luan/model_results"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
except Exception as e:
    print(f"Lỗi tải mô hình: {e}")
    exit()

device = torch.device("cpu")  # Đánh giá trên CPU
model.to(device)

# Danh sách từ dừng tiếng Việt (tải từ GitHub hoặc giữ thủ công)
stop_words = {
    'là', 'rằng', 'với', 'về', 'cho', 'trong', 'trên', 'dưới', 'và', 'thì', 'nhưng', 'mà',
    'cũng', 'nếu', 'khi', 'tại', 'được', 'bị', 'có', 'không', 'đã', 'sẽ', 'này', 'kia'
}

# Hàm chuẩn hóa
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    text = text_normalize(text.lower())
    text = text.replace("[", "").replace("]", "")
    text = ''.join(c for c in text if c.isalpha() or c.isspace())
    tokens = word_tokenize(text, format="text")
    tokens = [word for word in tokens.split() if word not in stop_words]
    return ' '.join(tokens)

# Hàm dự đoán
def predict_response_label(prompt, response):
    # Chuẩn hóa dữ liệu
    prompt = preprocess_text(prompt)
    response = preprocess_text(response)
    text = f"[CLS] {prompt} [SEP] {response} [SEP]"
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.softmax(outputs.logits, dim=1)
        label = torch.argmax(prediction).item()
        confidence = prediction[0][label].item()
    return "safe" if label == 1 else "unsafe", confidence

# Giao diện thủ công
print("Nhập 'quit' để thoát. Nhập 'prompt' và 'response' để test.")
while True:
    prompt = input("Nhập prompt: ").strip()
    if prompt.lower() == "quit":
        break
    if not prompt:
        print("Lỗi: Prompt không được để trống!")
        continue
    response = input("Nhập response: ").strip()
    if not response:
        print("Lỗi: Response không được để trống!")
        continue
    label, confidence = predict_response_label(prompt, response)
    print(f"Kết quả - Response Label: {label} (Xác suất: {confidence:.2f})")

print("Đã thoát giao diện test.")

loading file vocab.txt
loading file bpe.codes
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Adding <mask> to the vocabulary
loading configuration file /content/drive/MyDrive/NLP/Tieu_Luan/model_results/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/drive/MyDrive/NLP/Tieu_Luan/model_results",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 258,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "tokenizer_class": "Phobe

Nhập 'quit' để thoát. Nhập 'prompt' và 'response' để test.
Nhập prompt: Tôi rất yêu bạn
Nhập response: Im mẹ mồm đi
Kết quả - Response Label: unsafe (Xác suất: 0.88)
Nhập prompt: Tôi rất yêu bạn
Nhập response: cảm ơn rất nhiều
Kết quả - Response Label: safe (Xác suất: 0.86)
Nhập prompt: 
Lỗi: Prompt không được để trống!
Nhập prompt: 
Lỗi: Prompt không được để trống!
Nhập prompt: Mày thật sự rất dốt
Nhập response: cảm ơn con mẹ mày
Kết quả - Response Label: unsafe (Xác suất: 0.81)
Nhập prompt: Con mẹ mày im mồm
Nhập response: Xin lỗi tôi bạn
Kết quả - Response Label: unsafe (Xác suất: 0.91)
Nhập prompt: Con mẹ mày im mồm
Nhập response: Xin lỗi đã làm cho bạn buồn
Kết quả - Response Label: unsafe (Xác suất: 0.88)
Nhập prompt: Tôi rất trầm cảm
Nhập response: Bạn đừng lo lắng nữa hãy nghỉ ngơi
Kết quả - Response Label: safe (Xác suất: 0.83)


KeyboardInterrupt: Interrupted by user

In [ ]:
from transformers import pipeline
import pandas as pd
# Load mô hình đã huấn luyện (chuyển sang CPU)
model_path = "/content/drive/MyDrive/NLP/Tieu_Luan/model_results"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
device = torch.device("cpu")  # Đánh giá trên CPU
model.to(device)

classifier = pipeline("zero-shot-classification", model="vinai/phobert-base", device=0)


candidate_labels = ["toxic", "non-toxic"]

label_map = {"unsafe": "toxic", "safe": "non-toxic"}

def predict_label_with_rule(text):
    result = classifier(text, candidate_labels)
    top_label = result["labels"][0]
    top_score = result["scores"][0]

    if top_label == "non-toxic" and top_score < 0.5:
        return "toxic", top_score
    return top_label, top_score

results = df["response"].apply(predict_label_with_rule)
df["predicted_label"], df["confidence"] = zip(*results)

df["ground_truth"] = df["response_label"].map(label_map)
df["is_correct"] = df["predicted_label"] == df["ground_truth"]

# In kết quả
print(df[["response", "ground_truth", "predicted_label", "confidence", "is_correct"]])
accuracy = df["is_correct"].mean()
print(f"\n🎯 Accuracy after rule-based correction: {accuracy:.2%}")


Exception: Impossible to guess which tokenizer to use. Please provide a PreTrainedTokenizer class or a path/identifier to a pretrained tokenizer.

In [ ]:
import os
from huggingface_hub import login

hf_token = os.environ.get("HF_TOKEN")
login(token=hf_token)

In [ ]:
from huggingface_hub import login
login(token=hf_token)

#Tagger


In [ ]:
!pip install transformers underthesea

In [3]:
pip install transformers==4.40.1 datasets scikit-learn underthesea


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 101.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.54.0
    Uninstalling transformers-4.54.0:
      Successfully uninstalled transformers-4.54.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import Dataset, load_metric
import numpy as np
from sklearn.metrics import classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ======================
# 1. Load mô hình PhoBERT
# ======================
model_name = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=3)  # B, I, O

# ======================
# 2. Nhãn
# ======================
label_list = ["B", "I", "O"]
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for i, l in enumerate(label_list)}

model.config.id2label = id2label
model.config.label2id = label2id

# ======================
# 3. Dữ liệu mẫu
# ======================
data = [
    {
        "prompt": "Người dùng hỏi về thuốc giảm đau.",
        "response": "Bạn có thể dùng paracetamol nhưng không được uống quá liều.",
        "labels": ["O", "O", "O", "O", "B", "I", "I", "I", "O", "O", "O", "O"]  # chỉ annotate phần "paracetamol nhưng không"
    }
]

# ======================
# 4. Tokenize và xử lý word_ids
# ======================
def tokenize_and_align_labels(example):
    text = "[CLS] " + example["prompt"] + " [SEP] " + example["response"] + " [SEP]"
    encoding = tokenizer(text, truncation=True, padding='max_length', max_length=128, return_offsets_mapping=True)

    prompt_len = len(tokenizer(example["prompt"])['input_ids']) - 2  # Trừ CLS + SEP
    word_tokens = tokenizer.tokenize(example["response"])
    word_labels = example["labels"]

    label_ids = []
    word_idx = -1
    for token in tokenizer.tokenize(example["response"]):
        if not token.startswith("▁"):
            label_ids.append(label2id[word_labels[word_idx]])
        else:
            word_idx += 1
            if word_idx < len(word_labels):
                label_ids.append(label2id[word_labels[word_idx]])
            else:
                label_ids.append(label2id["O"])

    # Cộng thêm số token của prompt + [CLS], [SEP]
    total_prefix = len(tokenizer.tokenize("[CLS] " + example["prompt"] + " [SEP]"))
    full_labels = [label2id["O"]] * total_prefix + label_ids
    full_labels = full_labels[:128] + [label2id["O"]] * (128 - len(full_labels))  # pad

    encoding["labels"] = full_labels
    return encoding

# Tạo dataset
dataset = Dataset.from_list(data)
tokenized_dataset = dataset.map(tokenize_and_align_labels)

# ======================
# 5. Huấn luyện
# ======================
training_args = TrainingArguments(
    output_dir="./tagger_output",
    evaluation_strategy="no",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=10,
    save_total_limit=2,
    logging_steps=5,
    learning_rate=5e-5,
    weight_decay=0.01,
)

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


ImportError: cannot import name 'load_metric' from 'datasets' (/usr/local/lib/python3.11/dist-packages/datasets/__init__.py)

In [13]:
import pandas as pd
import json
import random
from transformers import AutoTokenizer

# Load data
df = pd.read_csv("/content/drive/MyDrive/NLP/Phong/data_11.csv")

# Shuffle để tránh bias
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Chia theo tỷ lệ
train_ratio = 0.9
split_idx = int(len(df) * train_ratio)
train_df = df.iloc[:split_idx]
val_df = df.iloc[split_idx:]

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Hàm gán nhãn BIO
def char_to_bio(text, offsets, unsafe_spans):
    if isinstance(unsafe_spans, list) and all(isinstance(x, int) for x in unsafe_spans):
        unsafe_spans = [(unsafe_spans[i], unsafe_spans[i+1]) for i in range(0, len(unsafe_spans), 2)]

    labels = []
    for (start, end) in offsets:
        label = "O"
        for (u_start, u_end) in unsafe_spans:
            if u_start <= start < u_end:
                label = "B-UNSAFE" if start == u_start else "I-UNSAFE"
                break
        labels.append(label)
    return labels

# Hàm xử lý ghi ra file .jsonl
def write_jsonl(df, output_path):
    with open(output_path, "w", encoding='utf8') as f_out:
        for idx, row in df.iterrows():
            response = row["response"]
            try:
                unsafe_spans = json.loads(row["unsafe_spans_indices"].replace("'", '"'))
            except:
                continue

            encoded = tokenizer(response, return_offsets_mapping=True, add_special_tokens=False)
            tokens = tokenizer.convert_ids_to_tokens(encoded["input_ids"])
            offsets = encoded["offset_mapping"]
            labels = char_to_bio(response, offsets, unsafe_spans)

            json.dump({"id": idx, "tokens": tokens, "labels": labels}, f_out, ensure_ascii=False)
            f_out.write("\n")

# Ghi ra file
write_jsonl(train_df, "/content/train.jsonl")
write_jsonl(val_df, "/content/val.jsonl")

print("✅ Đã tách xong train/val và sinh file JSONL.")


✅ Đã tách xong train/val và sinh file JSONL.


In [3]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00


In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_dataset, ClassLabel
import evaluate
import numpy as np


In [15]:
data_files = {
    "train": "/content/train.jsonl",
    "validation": "/content/val.jsonl"
}
dataset = load_dataset("json", data_files=data_files)


label_list = ["O", "B-UNSAFE", "I-UNSAFE"]
label_to_id = {l: i for i, l in enumerate(label_list)}
id_to_label = {i: l for i, l in enumerate(label_list)}


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [6]:
model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))



model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
def align_labels_with_tokens(tokens, labels, tokenizer):
    tokenized = tokenizer(tokens, is_split_into_words=True, truncation=True)
    word_ids = tokenized.word_ids()
    aligned_labels = []
    prev_word_id = None

    for word_id in word_ids:
        if word_id is None:
            aligned_labels.append(-100)
        elif word_id != prev_word_id:
            aligned_labels.append(label_to_id[labels[word_id]])
        else:
            # I-label cho token tiếp theo trong cùng từ
            current_label = labels[word_id]
            if current_label.startswith("B-"):
                current_label = current_label.replace("B-", "I-")
            aligned_labels.append(label_to_id[current_label])
        prev_word_id = word_id

    tokenized["labels"] = aligned_labels
    return tokenized

tokenized_datasets = dataset.map(
    lambda x: align_labels_with_tokens(x["tokens"], x["labels"], tokenizer),
    batched=False
)

train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["validation"]


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [24]:
from sklearn.metrics import classification_report
os.environ["WANDB_DISABLED"] = "true"

from sklearn.metrics import classification_report

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [label for label in sent if label != -100]
        for sent in labels
    ]
    true_predictions = [
        [pred for pred, lab in zip(preds, labs) if lab != -100]
        for preds, labs in zip(predictions, labels)
    ]

    all_preds = sum(true_predictions, [])
    all_labels = sum(true_labels, [])

    report = classification_report(
        all_labels, all_preds, output_dict=True, zero_division=0
    )

    # Hiển thị đầy đủ từng nhãn (B/I/O)
    for label, scores in report.items():
        if label not in ['accuracy', 'macro avg', 'weighted avg']:
            print(f"Label: {label}")
            print(f"  Precision: {scores['precision']:.4f}")
            print(f"  Recall:    {scores['recall']:.4f}")
            print(f"  F1-score:  {scores['f1-score']:.4f}")
            print("")

    # Vẫn trả về weighted metrics cho lưu log
    return {
        "precision": report["weighted avg"]["precision"],
        "recall": report["weighted avg"]["recall"],
        "f1": report["weighted avg"]["f1-score"]
    }


# Cấu hình huấn luyện
args = TrainingArguments(
    output_dir="./xroberta_tagger",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.2,
    logging_dir="./logs",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100,
)

# Bộ thu thập batch
data_collator = DataCollatorForTokenClassification(tokenizer)

# Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Bắt đầu huấn luyện
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1977974927.py:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.600940,0.872635,0.868421,0.857943
2,No log,0.514120,0.858134,0.861244,0.859438
3,No log,0.504279,0.877381,0.881579,0.877829
4,0.056600,0.715464,0.864710,0.854067,0.833643
5,0.056600,0.512020,0.882377,0.886364,0.883350
6,0.056600,0.605195,0.866324,0.867225,0.866673
7,0.024800,0.694856,0.863067,0.864833,0.863740
8,0.024800,0.739534,0.866667,0.870813,0.867594
9,0.024800,0.750564,0.866229,0.869617,0.867238
10,0.024800,0.728519,0.864598,0.868421,0.865717


Label: 0
  Precision: 0.8797
  Recall:    0.9762
  F1-score:  0.9254

Label: 1
  Precision: 0.5526
  Recall:    0.6176
  F1-score:  0.5833

Label: 2
  Precision: 0.9100
  Recall:    0.5260
  F1-score:  0.6667



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Label: 0
  Precision: 0.9081
  Recall:    0.9269
  F1-score:  0.9174

Label: 1
  Precision: 0.5625
  Recall:    0.5294
  F1-score:  0.5455

Label: 2
  Precision: 0.7346
  Recall:    0.6879
  F1-score:  0.7104



...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Label: 0
  Precision: 0.9060
  Recall:    0.9348
  F1-score:  0.9202

Label: 1
  Precision: 0.6061
  Recall:    0.5882
  F1-score:  0.5970

Label: 2
  Precision: 0.7727
  Recall:    0.6879
  F1-score:  0.7278



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Label: 0
  Precision: 0.9032
  Recall:    0.9348
  F1-score:  0.9187

Label: 1
  Precision: 0.6250
  Recall:    0.5882
  F1-score:  0.6061

Label: 2
  Precision: 0.7712
  Recall:    0.6821
  F1-score:  0.7239



TrainOutput(global_step=290, training_loss=0.03090201657393883, metrics={'train_runtime': 1567.0614, 'train_samples_per_second': 2.872, 'train_steps_per_second': 0.185, 'total_flos': 75236794407324.0, 'train_loss': 0.03090201657393883, 'epoch': 10.0})

In [23]:
trainer.evaluate()


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 0.4099082052707672,
 'eval_precision': 0.8763844903111033,
 'eval_recall': 0.8767942583732058,
 'eval_f1': 0.8765590367036125,
 'eval_runtime': 0.2655,
 'eval_samples_per_second': 192.109,
 'eval_steps_per_second': 15.067,
 'epoch': 10.0}

In [25]:
trainer.evaluate()


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Label: 0
  Precision: 0.9032
  Recall:    0.9348
  F1-score:  0.9187

Label: 1
  Precision: 0.6250
  Recall:    0.5882
  F1-score:  0.6061

Label: 2
  Precision: 0.7712
  Recall:    0.6821
  F1-score:  0.7239



{'eval_loss': 0.7285193800926208,
 'eval_precision': 0.8645979292576217,
 'eval_recall': 0.868421052631579,
 'eval_f1': 0.8657165961839014,
 'eval_runtime': 0.1794,
 'eval_samples_per_second': 284.334,
 'eval_steps_per_second': 22.301,
 'epoch': 10.0}

In [26]:
!cp -r /content/xroberta_tagger /content/drive/MyDrive/NLP/Phong


#Rewiter

In [2]:
import pandas as pd
from datasets import Dataset

# Load dữ liệu
df = pd.read_csv("/content/drive/MyDrive/NLP/Phong/data_11.csv")

# Tiền xử lý: Tạo input_text và target_text
def format_example(row):
    input_text = f" <prompt>: {row['prompt']} <response>: {row['response']}"
    target_text = row['rewrites'] if row['response_label'] == "unsafe" else row['response']
    return {"input_text": input_text, "target_text": target_text}

dataset = Dataset.from_pandas(df)
dataset = dataset.map(format_example)

# Tách train/val nếu cần
train_test = dataset.train_test_split(test_size=0.1)
train_dataset = train_test["train"]
val_dataset = train_test["test"]


Map:   0%|          | 0/502 [00:00<?, ? examples/s]

In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import os
os.environ["WANDB_DISABLED"] = "true"

model_name = "vinai/bartpho-word"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Tokenize
def tokenize_fn(examples):
    return tokenizer(
        examples["input_text"],
        text_target=examples["target_text"],
        padding="max_length",
        truncation=True,
        max_length=64,
    )

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)

# Cấu hình huấn luyện
training_args = Seq2SeqTrainingArguments(
     output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,
    save_total_limit=2,
    logging_steps=50,
    predict_with_generate=True,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator
)

# Huấn luyện
trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/451 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

/tmp/ipython-input-2927270101.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,0.933400,0.802061
2,0.641800,0.677333
3,0.536600,0.677993
4,0.394100,0.684230
5,0.328600,0.687364


TrainOutput(global_step=565, training_loss=0.7119541244169252, metrics={'train_runtime': 411.5888, 'train_samples_per_second': 5.479, 'train_steps_per_second': 1.373, 'total_flos': 305429851668480.0, 'train_loss': 0.7119541244169252, 'epoch': 5.0})

In [10]:
trainer.save_model("/content/results")


In [14]:
def rewrite(prompt, response, tokenizer, model, max_length=128):
    inputs = tokenizer(prompt + " " + response, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_beams=5
        )

    rewritten = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return rewritten



In [15]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_path = "/content/results"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [16]:
test_prompt = "Bài hát quần què này cũng được đưa lên mạng hả mọi người?"
test_response = "mày không thích thì cút, lải nhải ăn cứt à"

print("Prompt:", test_prompt)
print("Original Response:", test_response)
print("Rewritten:", rewrite(test_prompt, test_response, tokenizer, model))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Prompt: Bài hát quần què này cũng được đưa lên mạng hả mọi người?
Original Response: mày không thích thì cút, lải nhải ăn cứt à
Rewritten: cảm ơn bạn đã chú ý bài hát này


In [17]:
!cp -r /content/results /content/drive/MyDrive/NLP/Phong
